In [52]:
%run SetUpParams.ipynb

2024-08-15 02:37:39.483224
2004-08-15 02:37:39.483224


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler 
import torch 

### Create dataframe to feed into model

In [54]:
final_df = pd.DataFrame()
# Next Day Open Price - Current Day Open Price
final_df[f'{target_stock}_change'] = target_stock_data['Open'].shift(-1) - target_stock_data['Open']
final_df[f'{target_stock}_lag'] = final_df[f'{target_stock}_change'].shift(1)
final_df.reset_index(inplace=True)
print(final_df)

           Date  ^GSPC_change  ^GSPC_lag
0    2004-08-16     14.539917        NaN
1    2004-08-17      2.369995  14.539917
2    2004-08-18     13.460083   2.369995
3    2004-08-19     -3.940063  13.460083
4    2004-08-20      7.119995  -3.940063
...         ...           ...        ...
5029 2024-08-08     62.090332 -40.560059
5030 2024-08-09     37.219727  62.090332
5031 2024-08-12     25.100098  37.219727
5032 2024-08-13     65.379883  25.100098
5033 2024-08-14           NaN  65.379883

[5034 rows x 3 columns]


#### Add European and US Stock Market Price Changes to final_df
Current Day Open Price - Previous Day Open Price

In [55]:
# EU Stock Market has already opened
for ticker in US_EU_market_dict.keys():
        data = US_EU_market_dict[ticker]
        # Current Day Open Price - Previous Day Open Price
        temp = (data['Open'] - data['Open'].shift(1)).reset_index()
        temp.rename(columns={'Open':ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-16     14.539917        NaN         NaN         NaN        NaN   
1    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
2    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
3    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
4    2004-08-20      7.119995  -3.940063   -6.369995  -41.970703 -11.910156   
...         ...           ...        ...         ...         ...        ...   
5029 2024-08-08     62.090332 -40.560059 -214.041016 -289.710938  61.069824   
5030 2024-08-09     37.219727  62.090332  228.250000  467.679688  36.690430   
5031 2024-08-12     25.100098  37.219727  157.121094  147.953125  37.139648   
5032 2024-08-13     65.379883  25.100098  151.099609 -110.742188 -19.139648   
5033 2024-08-14           NaN  65.379883  282.859375  355.332031  52.459961   

          ^GDAXI  
0            NaN  
1      53.129

#### Add Asian Stock Market Price Changes to final_df
Close Price - Open Price

In [56]:
# Asian Stock Market has already closed; one day ahead of US Stock Market
for ticker in asia_market_dict.keys():
        data = asia_market_dict[ticker]
        # Close Price - Open Price
        temp = (data['Close'] - data['Open']).reset_index()
        temp.rename(columns={0:ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-16     14.539917        NaN         NaN         NaN        NaN   
1    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
2    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
3    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
4    2004-08-20      7.119995  -3.940063   -6.369995  -41.970703 -11.910156   
...         ...           ...        ...         ...         ...        ...   
5029 2024-08-08     62.090332 -40.560059 -214.041016 -289.710938  61.069824   
5030 2024-08-09     37.219727  62.090332  228.250000  467.679688  36.690430   
5031 2024-08-12     25.100098  37.219727  157.121094  147.953125  37.139648   
5032 2024-08-13     65.379883  25.100098  151.099609 -110.742188 -19.139648   
5033 2024-08-14           NaN  65.379883  282.859375  355.332031  52.459961   

          ^GDAXI       ^AORD        ^HSI       ^N22

In [57]:
# Different markets has different holidays
# Forward fill null values
final_df.ffill(inplace=True)
final_df.dropna(inplace=True, ignore_index=True)
# final_df.isna().sum()
final_df

,Date,^GSPC_change,^GSPC_lag,^IXIC,^DJI,^FCHI,^GDAXI,^AORD,^HSI,^N225,^NSEI
0,2007-09-17,-7.609985,0.290039,11.890137,20.560547,-16.479980,-17.530273,-23.299805,-378.080078,232.370117,-23.800293
1,2007-09-18,43.119995,-7.609985,-1.080078,-38.770508,-104.929688,-37.049805,-66.500000,73.019531,-235.690430,52.100098
2,2007-09-19,8.939941,43.119995,72.929932,337.430664,264.319824,169.069824,113.100098,43.281250,343.419922,182.100098
3,2007-09-20,-9.939941,8.939941,-6.309814,72.909180,2.810059,99.669922,14.299805,62.820312,-60.871094,12.699707
4,2007-09-21,7.000000,-9.939941,11.169922,-45.189453,-18.060059,-5.809570,-22.099609,183.818359,28.180664,84.599609
...,...,...,...,...,...,...,...,...,...,...,...
4252,2024-08-08,62.090332,-40.560059,-214.041016,-289.710938,61.069824,77.400391,-26.600098,118.791016,185.359375,-131.550781
4253,2024-08-09,37.219727,62.090332,228.250000,467.679688,36.690430,156.849609,104.200195,-7.128906,-247.339844,-19.349609
4254,2024-08-12,25.100098,37.219727,157.121094,147.953125,37.139648,111.128906,37.799805,8.669922,-247.339844,26.949219
4255,2024-08-13,65.379883,25.100098,151.099609,-110.742188,-19.139648,15.630859,13.700195,13.849609,741.933594,-203.349609


### Prepare Sequences

In [58]:
input_df = final_df.copy().drop(columns=[f'{target_stock}_change', 'Date'])

In [59]:
# Train-Test Split
proportion_test = 0.8
gap = 60

output_df = final_df[f'{target_stock}_change'].to_frame()

train_index = round(len(input_df) * proportion_test)
test_index = train_index + gap

X_train = input_df.iloc[:train_index]
X_test = input_df.iloc[test_index:]

y_train = output_df.iloc[:train_index]
y_test = output_df.iloc[test_index:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3406, 9)
(791, 9)
(3406, 1)
(791, 1)


In [60]:
# Scaling inputs using Normalisation
scaler = StandardScaler()
X_scaled_train_temp = scaler.fit_transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train_temp, columns = input_df.columns)

X_scaled_test_temp = scaler.fit_transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test_temp, columns = input_df.columns)
print(X_scaled_train)
print(X_scaled_test)

      ^GSPC_lag     ^IXIC      ^DJI     ^FCHI    ^GDAXI     ^AORD      ^HSI  \
0     -0.020091  0.120217  0.077882 -0.279588 -0.162336 -0.449984 -1.572381   
1     -0.394124 -0.056315 -0.232118 -1.761201 -0.325087 -1.267340  0.373283   
2      2.007731  0.951006  1.733507  4.424063  1.393503  2.130730  0.245017   
3      0.389447 -0.127495  0.351400  0.043538  0.814859  0.261408  0.329293   
4     -0.504438  0.110415 -0.265657 -0.306056 -0.064611 -0.427276  0.851176   
...         ...       ...       ...       ...       ...       ...       ...   
3401   0.124317  2.120837 -1.369944 -1.013274 -1.234244  0.660627  0.421722   
3402   0.966128  1.355506  0.439025 -0.297345  0.241470 -0.168075 -2.144127   
3403  -0.870432 -1.296369 -1.182265 -0.701542 -0.337097  0.507371 -2.181698   
3404  -1.955585 -6.093708 -0.677804  0.153924  0.040695  0.155463  1.229487   
3405   1.754906  2.019714  1.716939  1.138710  1.352812  0.757121  1.321142   

         ^N225     ^NSEI  
0     1.468232 -0.223839

In [61]:
def create_sequences(X_data, y_data, time_step = 10):
    X, y = [], []
    for i in range(len(X_data) - time_step):
        X.append(X_data.iloc[i:i + time_step, :].values)
        y.append(y_data.iloc[i+time_step, :].values)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [62]:
X_train_seq, y_train_seq = create_sequences(X_scaled_train, y_train)
X_test_seq, y_test_seq = create_sequences(X_scaled_test, y_test)

In [63]:
# Shape of Tensor = (Number of Sequences, Time Steps, Number of Features)
print(X_train_seq.shape)
print(y_train_seq.shape)
print(X_test_seq.shape)
print(y_test_seq.shape)

torch.Size([3396, 10, 9])
torch.Size([3396, 1])
torch.Size([781, 10, 9])
torch.Size([781, 1])
